In [1]:
import os
import shutil
import pandas as pd
from tqdm import tqdm      
import re
from pathlib import Path
import traceback
from os.path import join
import file_function as ff
import time
from datetime import datetime

### 나우리 나스 파일 옮기기

##### depth2,3 제거하고 파일명에 채무자키 없는 거 추가하기

In [7]:
# 문서구분 폴더를 기준으로 채무자키 파일명에 넣어서 파일 저장하기
path = r'D:\nas다운로드\나우리서버\법원서류'
path_dst = r'D:\nas다운로드\upload\법원서류'
path_s = r'C:\Users\SL\Desktop\test\log\success'
path_f = r'C:\Users\SL\Desktop\test\log\fail'
path_d = r'C:\Users\SL\Desktop\test\log\dir'

file_list = os.walk(path)
p_key = re.compile(r"[\d]{8}(?!\d)") #match


result_f = []
result_d = []
error = []
os.chdir(r'D:\\')

In [8]:
for root, dirs, files in tqdm(file_list) :
    # root : 삭제할 채무자키 폴더
    #os.chdir(root)
    for f in files :
        try : 
            n = f
            if not p_key.match(n) : # 파일명에 채무자키 없는 경우
                key = root.split(os.path.sep)[-1]
                # == key = os.path.split(root)[1]
                # print("key : ", key)
                n = key + " " + n
                # print("n : ", n)
            else : pass

            result_f.append(ff.re_name(join(root, f), join(path_dst, n)))
        except Exception as e:
            print("===================================")
            print(datetime.today().strftime("%H:%M:%S"))
            print(f, e.__class__, e, sep=" : ")
            print(traceback.format_exc())
            error.append([f, e.__class__, e])
        continue # 반복문 계속 돌아
    #os.chdir(r'D:\nas다운로드\나우리서버')

# for root, dirs, files in tqdm(os.walk(path)) :
#     for dir in dirs :
#         try : 
#             os.removedirs(dir)
#             result_d.append([dir]) #log는 리스트를 인자로 받는다.
#         except Exception as e:
#             print("===================================")
#             print(datetime.today().strftime("%H:%M:%S"))
#             print(root, e.__class__, e, sep=" : ")
#             print(traceback.format_exc())
#             error.append([dir, e.__class__, e])
#             continue # 반복문 계속 돌아


ff.write_log_csv(result_f, path_s)
ff.write_log_csv(error, path_f)
#ff.write_log_csv(result_d, path_d)
print(f"총 파일 : {len(result_f) + len(error)}, 옮긴 파일 수 : {len(result_f)}, 에러 파일 수 : {len(error)}")

# 남아있는 디렉토리 확인
# remain = []
# for d in tqdm(result_d, total=len(result_d)) :
#     if os.path.isdir(d) :
#         remain.append(d)
# if len(remain) >0 : print(len(remain))
# else : print("모든 디렉토리 삭제 완료")

6646it [02:15, 49.00it/s]

log 파일이 비어있습니다. path : C:\Users\SL\Desktop\test\log\fail
총 파일 : 9499, 옮긴 파일 수 : 9499, 에러 파일 수 : 0


##### 파일명에 문서구분 없는 거 별도 폴더로 옮기기

In [6]:
path = r'D:\nas다운로드\upload\양도통지서\nodocu'
nodocuPath = r'D:\nas다운로드\upload\양도통지서\nodocu' ####################만들어
##########################################
standard = "양도통지서"
####### pdf 아닌 경우, 수작업해야할 게 많은지 체크해서 그렇다면 내보내 ###########
p2=re.compile(r"양도\s?통지서?|채권\s?양도\s?통지서|(?<![^가-힣][가-힣])양통")  #_세양통신 해결pass
##########################################

p_extension = re.compile('(jpeg|jpg|bmp|gif|pdf|png|tif)$', re.I)
file_list = [f.name for f in os.scandir(path) if f.is_file() and (re.search(p_extension, f.name))]
total = len(file_list)

cnt = 0
for f in tqdm(file_list, total=total) :
    if p2.search(f) :
        n = p2.sub(standard, f) #관련단어 있는 것은 정식명칭으로 바꾼뒤 그대로 두고
        if f != n :
            os.rename(join(path, f), join(path, n))
    else : # 키워드 아예 없는 경우는 nodocu폴더로
        os.rename(join(path, f), join(nodocuPath, f))
        cnt += 1
print(f'{total}개 중 {cnt}개 nodou로 이동')

#########################
# 정식명으로 바꾼거는 nas에 올리면 돼
# nodocu에 있는 거는 다른 문서는 옮겨주고 쓸데없는 말 지우고 채무자키 이름 빼고 지워서 끝에 문서이름 넣어주는 걸로

100%|██████████| 894/894 [00:00<00:00, 3365.77it/s]

894개 중 29개 nodou로 이동


In [ ]:
##### 끝에 문서구분 붙이고 중간에 있는 일련번호 지우자

##### 구나스 등초본

In [19]:
path_s = r'C:\Users\SL\Desktop\test\log\success'
path_f = r'C:\Users\SL\Desktop\test\log\fail'
path_d = r'C:\Users\SL\Desktop\test\log\dir'
path = r'D:\nas다운로드\upload\등초본' #폴더만 있음

result_f = []
result_d = []
error = []

file_list = os.listdir(path)
total = len(file_list)
#temp_cwd = os.getcwd()
for dir in tqdm(file_list, total=total):
    
    miss_dir = join(path, dir) #파일명들어간 폴더
    
    # print("miss_dir : ",miss_dir)
    # print("dst_dir : ", dst_dir, type(dst_dir))
    # print("dst_f : ", dst_f)

    os.chdir(miss_dir)
    for f in os.listdir(miss_dir) :
        # print("f : ",f)
        if os.path.isfile(f) :
            try : 
                result_f.append(ff.re_name(f, join(path, dir)))
                
            except Exception as e:
                print("===================================")
                print(datetime.today().strftime("%H:%M:%S"))
                print(f, e.__class__, e, sep=" : ")
                print(traceback.format_exc())
                error.append([f, e.__class__, e])
                continue # 반복문 계속 돌아
    os.chdir(path)
    try : 
        os.rmdir(miss_dir)
        result_d.append(miss_dir)
    except Exception as e:
        print("===================================")
        print(datetime.today().strftime("%H:%M:%S"))
        print(f, e.__class__, e, sep=" : ")
        print(traceback.format_exc())
        error.append([f, e.__class__, e])
        continue # 반복문 계속 돌아

ff.write_log_csv(result_f, path_s)
ff.write_log_csv(result_d, path_d)
ff.write_log_csv(error, path_f)

# 남아있는 디렉토리 확인
remain = []
for d in tqdm(result_d, total=len(result_d)) :
    if os.path.isdir(d) :
        remain.append(d)
if len(remain) >0 : print(remain)
else : print("모든 디렉토리 삭제 완료")


100%|██████████| 2806/2806 [00:02<00:00, 1129.32it/s]


log 파일이 비어있습니다. path : C:\Users\SL\Desktop\test\log\fail


100%|██████████| 2806/2806 [00:00<00:00, 23505.62it/s]

모든 디렉토리 삭제 완료


##### 법원서류

In [9]:
path = r'C:\Users\SL\Desktop\test'
dst_path = r'D:\nas다운로드\upload\문서구분완료'
docu = " 집행권원 "

p_extension = re.compile('(jpeg|jpg|bmp|gif|pdf|png|tif)$', re.I)
file_list = [f.name for f in os.scandir(path) if f.is_file() and (re.search(p_extension, f.name))]
total = len(file_list)
cnt = 0

p_event1=re.compile(r"(?<=\D)([12][09]\d\d)([ㄱ-ㅎ가-힣]{1,3})\s([0-9]+)|(?<=\D)([12][09]\d\d)\s([ㄱ-ㅎ가-힣]{1,3})([0-9]+)")
p_event2=re.compile(r"(?<=\D)(\d{2})\s?([ㄱ-ㅎ가-힣]{1,3})\s?([0-9]+)")
p_event = re.compile(r"(?<=\D)\d{4}[ㄱ-ㅎ가-힣]{1,3}\d+")

p_rm = re.compile(r"TAA\(회\)|SCSB|[a-zA-Z\-\d_]{2,}_[a-zA-Z\-\d_]{4,}|\(?\s?[a-zA-Z][\d]{9,}\s?\)?|\(?\s?[a-zA-Z\d]{2,}-[a-zA-Z\d]{2,}-[a-zA-Z\d]{2,}\s?\)?|\(?\s?[a-zA-Z0-9]{4,}-[a-zA-Z0-9]{4,}\s?\)?|\(?\s?\d{10,}\s?\)?|(?<![가-힣\d])[13-9]\d{5,}(?!가-힣\d])|(?<![가-힣\d])\d{3,4}(?![가-힣\d])|(?<![가-힣\d])\d{6}[\-_]\d{6}(?!가-힣\d])|(-)?\d{1,3}(,\d{3})+(\.\d+)?")
#                                 언더바로 연결된 영어,숫자 키(C1_3284792)                         2개의 하이픈으로 연결된 영어 및 숫자          하이픈 또는 언더바로 연결된 4자리 이상의 영숫자(주민번호 포함)  10자리키          날짜 아닐 6자리 이상 단독 숫자      3~4자리의 단독 숫자                 앞과 교집합있지만, 확실한 주민번호       금액
p_sign = re.compile("[^a-zA-Zㄱ-ㅎ가-힣0-9\s_()]|\(\s?\)|_ | _|  ")

list = []

In [10]:
for f in tqdm(file_list, total=total):
    n = os.path.splitext(f)[0]
    ext = os.path.splitext(f)[1]

    if p_event1.search(n):
        n=p_event1.sub(r'\1\2\3', n)
    elif p_event2.search(n):
        n=p_event2.sub(r'20\1\2\3', n)
    else : pass 

    if p_event.search(n) :
        e = p_event.search(n)
        e_s = e.start()
        e_e = e.end()

        n = n[:e_s] + docu + e.group()

        n = n[:8] + p_rm.sub(" ", n[8:]) 
        # 기호정리, 연속공백 : set임 순서 주의
        n = p_sign.sub(" ", n)
        n = re.sub("[\s]{2,}", " ", n)

        new_f = n + ext

        list.append(ff.re_name(join(path, f), join(dst_path, new_f)))

        cnt += 1

ff.write_log_csv(list, path)
print(f'{cnt}개 이동 완료')

100%|██████████| 185/185 [00:00<00:00, 477.91it/s]

168개 이동 완료


In [ ]:
for f in tqdm(file_list, total=total):
    n = os.path.splitext(f)[0]
    ext = os.path.splitext(f)[1]

    

### 잘못 만든 상위 폴더 지우기

##### 부산 > nas

In [60]:
df = pd.read_excel(r"C:\Users\SL\Desktop\통합 문서1.xlsx", names=['src_dir', 'src_f', 'dst_f', 'dst_dir'], header=None)
total = len(df.index)
print(total)
path_s = r'C:\Users\SL\Desktop\test\log\success'
path_f = r'C:\Users\SL\Desktop\test\log\fail'
path_d = r'C:\Users\SL\Desktop\test\log\dir'

result_f = []
result_d = []
error = []

#temp_cwd = os.getcwd()
for key, row in tqdm(df.iterrows(), total=total):
    dst_dir = row.dst_dir
    dst_f = row.dst_f
    miss_dir = (join(dst_dir, dst_f)) #파일명들어간 폴더
    
    # print("miss_dir : ",miss_dir)
    # print("dst_dir : ", dst_dir)

    for f in os.listdir(miss_dir) :
        os.chdir(miss_dir)
        # print("f : ",f)
        if os.path.isfile(f) :
            try : 
                result_f.append(ff.re_name(f, join(dst_dir, f)))
                
            except Exception as e:
                print("===================================")
                print(datetime.today().strftime("%H:%M:%S"))
                print(f, e.__class__, e, sep=" : ")
                print(traceback.format_exc())
                error.append([f, e.__class__, e])
                continue # 반복문 계속 돌아
    os.chdir(dst_dir)
    try : 
        os.rmdir(miss_dir)
        result_d.append(miss_dir)
    except Exception as e:
        print("===================================")
        print(datetime.today().strftime("%H:%M:%S"))
        print(f, e.__class__, e, sep=" : ")
        print(traceback.format_exc())
        error.append([f, e.__class__, e])
        continue # 반복문 계속 돌아

ff.write_log_csv(result_f, path_s)
ff.write_log_csv(result_d, path_d)
ff.write_log_csv(error, path_f)

# 남아있는 디렉토리 확인
remain = []
for d in tqdm(result_d, total=len(result_d)) :
    if os.path.isdir(d) :
        remain.append(d)
if len(remain) >0 : print(remain)
else : print("모든 디렉토리 삭제 완료")


2671
